In [2]:
import pandas as pd
from datetime import timedelta

In [3]:
sales = pd.read_excel('./Online Retail.xlsx')



In [4]:
sales.columns

Index(['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'UnitPrice', 'CustomerID', 'Country'],
      dtype='object')

In [5]:
# bỏ một số dòng không dùng tới
sales.dropna(subset=['CustomerID'], inplace=True)

In [6]:
# tính tổng giá từng dòng
sales['Price'] = sales['Quantity'] * sales['UnitPrice']

In [7]:
# tổng hợp doanh số bán hàng của chúng tôi ở cấp độ đơn đặt hàng
orders = sales.groupby(['InvoiceNo', 'InvoiceDate', 'CustomerID']).agg({'Price': lambda x: x.sum()}).reset_index()
orders.head()

,InvoiceNo,InvoiceDate,CustomerID,Price
0,536365,2010-12-01 08:26:00,17850.0,139.12
1,536366,2010-12-01 08:28:00,17850.0,22.20
2,536367,2010-12-01 08:34:00,13047.0,278.73
3,536368,2010-12-01 08:34:00,13047.0,70.05
4,536369,2010-12-01 08:35:00,13047.0,17.85


In [8]:
#Ngày Now này sẽ được dùng làm tham chiếu để tính Recencyđiểm.
NOW = orders['InvoiceDate'].max() + timedelta(days=1)
#thời gian tìm hiểu là 1 năm
period = 365

In [9]:
# tìm giá trị tối thiểu của cột này cho mỗi khách hàng
orders['DaysSinceOrder'] = orders['InvoiceDate'].apply(lambda x: (NOW - x).days)

In [10]:
aggr = {
    'DaysSinceOrder': lambda x: x.min(),  # the number of days since last order (Recency)
    'InvoiceDate': lambda x: len([d for d in x if d >= NOW - timedelta(days=period)]), # the total number of orders in the last period (Frequency)
}
rfm = orders.groupby('CustomerID').agg(aggr).reset_index()
rfm.rename(columns={'DaysSinceOrder': 'Recency', 'InvoiceDate': 'Frequency'}, inplace=True)
rfm.head()

,CustomerID,Recency,Frequency
0,12346.0,326,2
1,12347.0,2,6
2,12348.0,75,4
3,12349.0,19,1
4,12350.0,310,1


In [11]:
# cộng Giá trị tiền tệ của từng khách hàng bằng cách cộng doanh số bán hàng trong năm qua
rfm['Monetary'] = rfm['CustomerID'].apply(lambda x: orders[(orders['CustomerID'] == x) & \
                                                           (orders['InvoiceDate'] >= NOW - timedelta(days=period))]\
                                                           ['Price'].sum())
rfm.head()

,CustomerID,Recency,Frequency,Monetary
0,12346.0,326,2,0.00
1,12347.0,2,6,3598.21
2,12348.0,75,4,1797.24
3,12349.0,19,1,1757.55
4,12350.0,310,1,334.40


In [12]:
#tôi lấy các nhóm ngũ vị phân cho từng tham số
quintiles = rfm[['Recency', 'Frequency', 'Monetary']].quantile([.2, .4, .6, .8]).to_dict()
quintiles

{'Recency': {0.2: 11.0, 0.4: 32.0, 0.6: 71.0, 0.8: 178.80000000000018},
 'Frequency': {0.2: 1.0, 0.4: 2.0, 0.6: 3.0, 0.8: 6.0},
 'Monetary': {0.2: 215.89800000000002,
  0.4: 440.4320000000001,
  0.6: 876.3679999999999,
  0.8: 1909.6580000000006}}

In [13]:
 #phương thức để gán thứ hạng từ 1 đến 5
def r_score(x):
    if x <= quintiles['Recency'][.2]:
        return 5
    elif x <= quintiles['Recency'][.4]:
        return 4
    elif x <= quintiles['Recency'][.6]:
        return 3
    elif x <= quintiles['Recency'][.8]:
        return 2
    else:
        return 1

def fm_score(x, c):
    if x <= quintiles[c][.2]:
        return 1
    elif x <= quintiles[c][.4]:
        return 2
    elif x <= quintiles[c][.6]:
        return 3
    elif x <= quintiles[c][.8]:
        return 4
    else:
        return 5    

In [14]:
rfm['R'] = rfm['Recency'].apply(lambda x: r_score(x))
rfm['F'] = rfm['Frequency'].apply(lambda x: fm_score(x, 'Frequency'))
rfm['M'] = rfm['Monetary'].apply(lambda x: fm_score(x, 'Monetary'))

In [15]:
rfm['RFM Score'] = rfm['R'].map(str) + rfm['F'].map(str) + rfm['M'].map(str)
# chia 365 ngày thành 12 để dễ dàng phân tích

for index, row in rfm.iterrows():
    if 1 <= row['Recency'] <= 31:
        rfm.at[index, 'Recency'] = 1
    elif 32 <= row['Recency'] <= 59:
        rfm.at[index, 'Recency'] = 2
    elif 60 <= row['Recency'] <= 90:
        rfm.at[index, 'Recency'] = 3
    elif 91 <= row['Recency'] <= 120:
        rfm.at[index, 'Recency'] = 4
    elif 121 <= row['Recency'] <= 151:
        rfm.at[index, 'Recency'] = 5
    elif 152 <= row['Recency'] <= 181:
        rfm.at[index, 'Recency'] = 6
    elif 182 <= row['Recency'] <= 212:
        rfm.at[index, 'Recency'] = 7
    elif 213 <= row['Recency'] <= 243:
        rfm.at[index, 'Recency'] = 8
    elif 244 <= row['Recency'] <= 273:
        rfm.at[index, 'Recency'] = 9
    elif 274 <= row['Recency'] <= 304:
        rfm.at[index, 'Recency'] = 10
    elif 305 <= row['Recency'] <= 334:
        rfm.at[index, 'Recency'] = 11
    else:
        rfm.at[index, 'Recency'] = 12
        
       


In [16]:
segt_map = {
    r'[1-2][1-2]': 'hibernating',
    r'[1-2][3-4]': 'at risk',
    r'[1-2]5': 'can\'t loose',
    r'3[1-2]': 'about to sleep',
    r'33': 'need attention',
    r'[3-4][4-5]': 'loyal customers',
    r'41': 'promising',
    r'51': 'new customers',
    r'[4-5][2-3]': 'potential loyalists',
    r'5[4-5]': 'champions'
}

rfm['Segment'] = rfm['R'].map(str) + rfm['F'].map(str)
rfm['Segment'] = rfm['Segment'].replace(segt_map, regex=True)
rfm.head()

,CustomerID,Recency,Frequency,Monetary,R,F,M,RFM Score,Segment
0,12346.0,11,2,0.00,1,2,1,121,hibernating
1,12347.0,1,6,3598.21,5,4,5,545,champions
2,12348.0,3,4,1797.24,2,4,4,244,at risk
3,12349.0,1,1,1757.55,4,1,4,414,promising
4,12350.0,11,1,334.40,1,1,2,112,hibernating


In [19]:

rfm.head(100)

,CustomerID,Recency,Frequency,Monetary,R,F,M,RFM Score,Segment
0,12346.0,11,2,0.00,1,2,1,121,hibernating
1,12347.0,1,6,3598.21,5,4,5,545,champions
2,12348.0,3,4,1797.24,2,4,4,244,at risk
3,12349.0,1,1,1757.55,4,1,4,414,promising
4,12350.0,11,1,334.40,1,1,2,112,hibernating
...,...,...,...,...,...,...,...,...,...
95,12462.0,1,4,1124.01,5,4,4,544,champions
96,12463.0,2,7,1303.63,3,5,4,354,loyal customers
97,12464.0,1,8,1212.05,5,5,4,554,champions
98,12465.0,1,3,733.89,5,3,3,533,potential loyalists


In [18]:
# lưu thành file csv để trực quan hóa dữ liệu bằng Power BI
rfm.to_csv('RFM.csv')
